In [2]:
import os
import ast
import pandas as pd

In [3]:
DATA_PATH = "original" 

In [4]:
refs_bp = pd.read_csv(f"{DATA_PATH}/TR0-TS0.csv", index_col="id")
refs_bp

,sequence,base_pairs
id,,
bpRNA_RFAM_37268,CGCGACGGCGAGCAAUCGCCAGCGUGACUGAUCAGGAGUCAGCAGA...,"[[2, 25], [4, 23], [7, 20], [8, 19], [9, 18], ..."
bpRNA_RFAM_12641,CUCUGGUAACUAGAGAUCCCUCAGACCCUUUUAGUCAGUGUGAAAA...,"[[2, 98], [3, 97], [4, 96], [5, 95], [6, 94], ..."
bpRNA_RFAM_21314,UUUAUGGGAUACACCAAGUUCAGUUUUGUCGCAUUCCACGAGCAUC...,"[[2, 69], [3, 68], [4, 67], [5, 66], [6, 65], ..."
bpRNA_CRW_42535,GGGCCGGUAGCUCAUUUAGGCAGAGCGUCUGACUCUUAAUCAGACG...,"[[1, 73], [2, 72], [3, 71], [4, 70], [5, 69], ..."
bpRNA_RFAM_11145,GACAACUGCUAAGUGAUUUAGGUAGUUUUAUGUUGUUGGGCUCUAU...,"[[5, 89], [6, 88], [7, 87], [8, 86], [13, 81],..."
...,...,...
CP007653.1_1278983-1278851,UUCUUUCCAAACGAUUUGAAGUUGCAGAACCUGAAAUGGCAGAUAG...,"[[2, 132], [3, 131], [4, 130], [5, 129], [6, 1..."
URS0000D68500_12908_1-68,AAAAAGUAGAAACCAGAAACUACUAAAAAAACGGGGAUAAAUCCGA...,"[[6, 23], [7, 22], [8, 21], [9, 20], [35, 52],..."
URS0000D6B7C1_12908_1-41,CCGUUCAACUCGCCCCUCCCCGGACGAGGGGCGUACGGGAG,"[[12, 32], [13, 31], [14, 30], [15, 29], [16, ..."


In [4]:
predictions = pd.read_csv(f"{RESULTS_PATH}TR0-TS0.csv", index_col="id")
predictions

,folding,method
id,,
bpRNA_RFAM_4878,.....((((((....))))))((((............)))).....,MXfold2
bpRNA_RFAM_21093,...............(.(((.(.........(((....)))........,MXfold2
bpRNA_RFAM_27096,..............................(((((((((...((((...,MXfold2
bpRNA_CRW_54070,(((((((........(((.......((((.((((...............,MXfold2
bpRNA_RFAM_19896,...((((...(((.(((((.(..(((((((((((.((..((((..(...,MXfold2
...,...,...
bpRNA_RFAM_10893,(((((((....))))))).....................(.(((((...,LinearPartition-C
bpRNA_RFAM_22060,....((((((((((((((((((((.....)))))))))))))))))...,LinearPartition-C
bpRNA_RFAM_36491,....((((((......(.(((((((((((......)))))))))))...,LinearPartition-C


In [5]:
def f1_strict(ref_bp, pre_bp):
    """F1 score strict, same as triangular but less efficient"""
    # corner case when there are no positives
    if len(ref_bp) == 0 and len(pre_bp) == 0:
        return 1.0, 1.0, 1.0

    tp1 = 0
    for rbp in ref_bp:
        if rbp in pre_bp:
            tp1 = tp1 + 1
    tp2 = 0
    for pbp in pre_bp:
        if pbp in ref_bp:
            tp2 = tp2 + 1

    fn = len(ref_bp) - tp1
    fp = len(pre_bp) - tp1

    tpr = pre = f1 = 0.0
    if tp1 + fn > 0:
        tpr = tp1 / float(tp1 + fn)  # sensitivity (=recall =power)
    if tp1 + fp > 0:
        pre = tp2 / float(tp1 + fp)  # precision (=ppv)
    if tpr + pre > 0:
        f1 = 2 * pre * tpr / (pre + tpr)  # F1 score

    return tpr, pre, f1


# All possible matching brackets for base pairing
MATCHING_BRACKETS = [
    ["(", ")"],
    ["[", "]"],
    ["{", "}"],
    ["<", ">"],
    ["A", "a"],
    ["B", "a"],
]
# Normalization.
BRACKET_DICT = {"!": "A", "?": "a", "C": "B", "D": "b"}

def fold2bp(struc, xop="(", xcl=")"):
    """Get base pairs from one page folding (using only one type of brackets).
    BP are 1-indexed"""
    openxs = []
    bps = []
    if struc.count(xop) != struc.count(xcl):
        return False
    for i, x in enumerate(struc):
        if x == xop:
            openxs.append(i)
        elif x == xcl:
            if len(openxs) > 0:
                bps.append([openxs.pop() + 1, i + 1])
            else:
                return False
    return bps

def dot2bp(struc):
    bp = []
    if not set(struc).issubset(
        set(["."] + [c for par in MATCHING_BRACKETS for c in par])
    ):
        return False

    for brackets in MATCHING_BRACKETS:
        if brackets[0] in struc:
            bpk = fold2bp(struc, brackets[0], brackets[1])
            if bpk:
                bp = bp + bpk
            else:
                return False
    return list(sorted(bp))

In [5]:
def get_motif_1nts_OLD(st_file, motif_id="S"):
    
    data = pd.read_csv(st_file, skiprows=7, index_col=0, delimiter=' ', usecols=[0,1], header=None)
    #print(data.head)

    # extract all the numeric ranges in column 1 if the index start with the character "S"
    dataS = data[data.index.str.startswith(motif_id)].reset_index()
    #print(dataS)

    ranges = dataS[1].str.extractall(r'(\d+)..(\d+)').astype(int)
    #print(ranges)

    # create a set with all nucleotides in these ranges
    nts_in_ranges = set()
    for start, end in ranges.values:
        nts_in_ranges.update(range(start, end+1))

    return nts_in_ranges
# test
#init_nt_set = get_motf_1nts("bpRNA_CRW_54567.st", "S")
#init_nt_set

In [ ]:
def get_motif_1nts(st_file, motif_id="S"):
    
    ranges = []
    with open(st_file, 'r') as file:
        rlines = 0
        for line in file:
            if not line.startswith('#'):
                rlines = rlines + 1
                #print(rlines, line)
                if rlines > 4 and line.startswith(motif_id):
                    if line.startswith(motif_id):
                        parts = line.split()
                        start = int(parts[1].split('..')[0])
                        end = int(parts[1].split('..')[1])
                        ranges.append((start, end))

    nts_in_ranges = set()
    for start, end in ranges:
        nts_in_ranges.update(range(start, end+1))

    return nts_in_ranges

# test
filename = "original/seqs/5s_Acanthamoeba-castellanii-1.st"
motif_id = 'S'

print(get_motif_1nts(filename, motif_id))
print(get_motif_1nts_OLD(filename, motif_id))

{1, 2, 3, 4, 5, 6, 7, 8, 9, 14, 15, 16, 17, 18, 19, 20, 21, 27, 28, 29, 30, 31, 32, 67, 68, 69, 70, 71, 78, 79, 80, 81, 82, 84, 85, 86}
{1, 2, 3, 4, 5, 6, 7, 8, 9, 14, 15, 16, 17, 18, 19, 20, 21, 27, 28, 29, 30, 31, 32, 67, 68, 69, 70, 71, 78, 79, 80, 81, 82, 84, 85, 86}


In [38]:
df[df['id'] == '5s_Acanthamoeba-castellanii-1']['motivos'].item()

'SSSSSSSSSMMMMSSSSSSSSIIIIISSSSSSHHHHHHHHHHHHSSSSBBSSIIIISSSSSSBSSMSSSSSIIIIIISSSSSBSSSHHHHSSSSSSSSIIIIISSSSSMSSSSSSSSSE'

In [13]:
df = pd.read_csv('original/ArchiveII_with_motiv.csv')
# Obtener la secuencia de motivos para la fila de interés
motivos_seq = df[df['id'] == '5s_Acanthamoeba-castellanii-1']['motivos'].values[0]

# Encontrar los índices donde aparece el motivo 'S'
indices_S = {i for i, char in enumerate(motivos_seq) if char == 'S'}

print("Índices de 'S' en la secuencia: ", indices_S)


Índices de 'S' en la secuencia:  {0, 1, 2, 3, 4, 5, 6, 7, 8, 13, 14, 15, 16, 17, 18, 19, 20, 26, 27, 28, 29, 30, 31, 44, 45, 46, 47, 50, 51, 56, 57, 58, 59, 60, 61, 63, 64, 66, 67, 68, 69, 70, 77, 78, 79, 80, 81, 83, 84, 85, 90, 91, 92, 93, 94, 95, 96, 97, 103, 104, 105, 106, 107, 109, 110, 111, 112, 113, 114, 115, 116, 117}


In [33]:
print(df[df['id'] == 'tRNA_tdbR00000127-Halocynthia_roretzi-7729-Gly-UCU'].sequence.item())
print(df[df['id'] == 'tRNA_tdbR00000127-Halocynthia_roretzi-7729-Gly-UCU'].structure.item())

GCUGUGUUAGUAUAAAGUAAUAUAUGUGAUUUCUAAUCAUGGGAUCCUUUAGGGACGUAGUACCA
(((((((..((((......)))).(((((.......)))))...((((..)))))))))))....


In [15]:
x = get_motif_1nts_OLD(filename, motif_id)
len(x)

36

In [16]:
len(indices_S)

72

In [8]:
def f1_motif(preds, refs, motif_id="S"):
    # mean F1 score for a given motif

    f1s = []
    for seq_id, row in preds.iterrows():
        # get the reference structure file
        ref_st_file = f"{DATA_PATH}bprna/TS0/{seq_id}.st"

        if os.path.exists(ref_st_file):

            # get the nucleotides in the motif
            nts_in_motif = get_motif_1nts(ref_st_file, motif_id)

            ref_bp = ast.literal_eval(refs_bp.loc[seq_id, "base_pairs"])
            
            folding = row["folding"]
            if folding[0] in {'.', '(', ')'}:
                pred_bp = dot2bp(folding)
            else:
                pred_bp = ast.literal_eval(folding)
            
            _, _, f1_full = f1_strict(ref_bp, pred_bp)
            #print(seq_id,f1_full)

            # remove base pairs with the first nucleotide not in nts_in_motif
            ref_bp_motif = [bp for bp in ref_bp if bp[0] in nts_in_motif]
            pred_bp_motif = [bp for bp in pred_bp if bp[0] in nts_in_motif]

            _, _, f1_motif = f1_strict(ref_bp_motif, pred_bp_motif)
            f1s.append(f1_motif)
            #print(seq_id,f1_motif)
    
        else:
            print("File does not exist:", ref_st_file)
    
    return sum(f1s) / len(f1s)

# test
preds_bpXXX = predictions.iloc[[1,100,2001],:]
f1_motif(preds_bpXXX, refs_bp, "S")

0.6244160130298745

In [12]:
# from https://github.com/hendrixlab/bpRNA/blob/master/bpRNA.pl#L569
# paired "Stem"     S
# Multiloop 	M
# Internal loop     I
# Bulge 	        B
# Hairpin loop      H
# pseudoKnot	K <<<<<<<<<<<<<<<< in .st files it is "PK" and pseudoKnots are in a different format
# dangling End	E
# eXternal loop     X

methods = predictions["method"].unique()
#motifs = ['B', 'E', 'H', 'I', 'S', 'X', 'M', 'K']
motifs = ['S', 'M', 'I', 'B', 'H', 'E', 'X']
# 
for motif_id in motifs:
    print(motif_id)
    for method in methods:
        preds = predictions.loc[predictions["method"] == method]
        print(method, f1_motif(preds, refs_bp, motif_id))

S
MXfold2 0.5938106155190924
UFold 0.715602438598144
sincFold 0.6877777093556202
RNAfold 0.6577189894406151
REDfold 0.699317276122512
LinearPartition-C 0.6697658957124076
M
MXfold2 0.7310344827586207
UFold 0.7072419106317411
sincFold 0.8551724137931035
RNAfold 0.653639846743295
REDfold 0.781055900621118
LinearPartition-C 0.6942528735632184
I
MXfold2 0.5134099616858238
UFold 0.41217257318952233
sincFold 0.6344827586206897
RNAfold 0.42605363984674327
REDfold 0.484472049689441
LinearPartition-C 0.4574712643678161
B
MXfold2 0.7808429118773946
UFold 0.697226502311248
sincFold 0.8605363984674329
RNAfold 0.7195402298850575
REDfold 0.796583850931677
LinearPartition-C 0.7547892720306514
H
MXfold2 0.4559386973180077
UFold 0.2781201848998459
sincFold 0.5992337164750958
RNAfold 0.2842911877394636
REDfold 0.48990683229813664
LinearPartition-C 0.3563218390804598
E
MXfold2 0.6835249042145594
UFold 0.5238828967642527
sincFold 0.7639846743295019
RNAfold 0.41379310344827586
REDfold 0.6816770186335404
Li